# A finite set of sufficient conditions for trumping between two state vectors:
Let $n> 1$, $\mathbf x,\mathbf y\in P_n$, $\mathbf x$ have weight $n$ and choose
\begin{align}
r=\frac{\log n}{\log y_1-\log x_1}. \nonumber
\end{align}

Here we say that $\mathbf y$ majorizes $\mathbf x$ if following conditions are satisfied.

1. For $r>0$ and $\bar r=\lfloor r+1\rfloor$,
\begin{align}
F_{k,\bar r}(\mathbf x)> F_{k,\bar r}(\mathbf y), \qquad \bar r\leq k\leq n\bar r,
\end{align}

2. $H_1(\mathbf x)>H_1(\mathbf y)$,\
and either \\
3.
 (a) the weight of $\mathbf y$ is smaller than $n$, \
 or,\
 (b) the weight of $\mathbf y$ is $n$, $s>0$ with
 \begin{align}
 s=\frac{\log n}{\log x_{\min}-\log y_{\min}} \nonumber
 \end{align}
and
\begin{align}
F_{k,1}\left(\frac{1}{\mathbf x^{\bar s}}\right)< F_{k,1}\left(\frac{1}{\mathbf y^{\bar s}}\right), \qquad 1\leq k\leq n \ \nonumber.
\end{align}

Here the symmetric polynomials $F_{k,r}:\mathbb R^n\mapsto \mathbb R$ indexed by $k,r\in\mathbb N$:
\begin{equation}
	F_{k,r}(\mathbf x) = \sum_{\substack{(k_1,\ldots,k_n)\text{ s.t. }\\\sum_ik_i=k, \max k_i\leq r}}\prod_{i=1}^n \frac{\left(x_i\right)^{k_i}}{k_i!} .
\end{equation}
and
\begin{equation}
H_1(\mathbf x) := -\sum_{i=1}^nx_i\log x_i .
\end{equation}

In [ ]:
# Checking the conditions for existence of catalysis for two general state vectors

import math
from itertools import product

# For rearranging the list in descending order
def descending_order(list_vector):
    list_vector.sort(reverse=True)
    return list_vector

# maximum number of elements of a vector
def compute_n(vector_a, vector_b):
    return max(len(vector_a), len(vector_b))

# For checking the first condition

def compute_r(x, y):
    return math.log(compute_n(x, y)) / (math.log(descending_order(y)[0]) -\
           math.log(descending_order(x)[0]))

def F(k, r, x):
    n = compute_n(x, y)  # Number of elements in the list
    result = 0.0

    # Generate all possible combinations of (k1, k2, ..., kn) with 1 <= ki <= r
    for k_comb in product(range(1, r + 1), repeat=n):

        if sum(k_comb) == k:
            product_term = math.prod([x[i]**k_comb[i] / math.factorial(k_comb[i]) for i in range(n)])
            result += product_term
    return result

def check_condition_1(x, y):
    r_new = math.floor(compute_r(x, y) + 1)
    n_new = compute_n(x, y)
    k_max = n_new * r_new

    for k in range(r_new, k_max + 1):
        Fx = F(k, r_new, x)
        Fy = F(k, r_new, y)
        if Fx < Fy:
            return "Trumping not possible"
    return "Condition 1 satisfied"

# For checking the second condition
def H1(x):
    result = 0
    for xi in x:
        if xi > 0:  # Avoid log(0) which is undefined
            result += -xi * math.log(xi)
    return result

def check_condition_2(x, y):
    if H1(x) < H1(y):
        return "Trumping not possible"
    return "condition 2 satisfied"

# For checking the third condition

def reciprocal_vector(x, s):
    return [1 / xi**s if xi != 0 else float('inf') for xi in x]

def compute_s(n, x, y):
    log_xmin = math.log(min(x))
    log_ymin = math.log(min(y))
    s = math.log(n) / (log_xmin - log_ymin)
    return s

def check_condition_3(x, y):
    s = max(0, math.floor(compute_s(len(x), x, y)+1))

    for k in range(1, len(x) + 1):
        F_x = F(k, 1, reciprocal_vector(x, s))
        F_y = F(k, 1, reciprocal_vector(y, s))
        if F_x > F_y:
            return "Trumping not possible"

    return "condition 3 satisfied"

# Example usage
x = [0.61, 0.3045, 0.0435, 0.042]
y = [0.7315, 0.1211, 0.1374, 0.01]

#x = [1.22472, 0.611838, 0.0874053, 0.0843912]
#y = [1.32971, 0.220307, 0.24996, 0.0181921]

result = check_condition_1(x, y)
print("Result:", result)

result = check_condition_2(x, y)
print("Result:", result)

result = check_condition_3(x, y)
print("Result:", result)

Result: Condition 1 satisfied
Result: condition 2 satisfied
Result: condition 3 satisfied


**# This part is same as above but using sympy**

In [ ]:
from fractions import Fraction
from functools import reduce
from itertools import product
import sympy as sp
import math

# For rearranging the list in descending order
def descending_order(list_vector):
    list_vector.sort(reverse=True)
    return list_vector

# maximum number of elements of a vector
def compute_n(vector_a, vector_b):
    return max(len(vector_a), len(vector_b))

# For checking the first condition
def compute_r(x, y):
    return math.log(compute_n(x, y)) / (math.log(descending_order(y)[0]) -\
           math.log(descending_order(x)[0]))

def Pr(x_i, t, r):
    return sum([(x_i * t)**k / sp.factorial(k) for k in range(1, r + 1)])

def polynomial_product(x, t, r):
    product = 1
    for x_i in x:
        product *= Pr(x_i, t, r)
    return sp.expand(product)

def collect_coefficient(polynomial, t, k):
    return polynomial.coeff(t, k)

def F(k, r, x):
    n = compute_n(x, y)  # Number of elements in the list
    t = sp.symbols('t')
    polynomial = polynomial_product(x, t, r)
    coefficient = collect_coefficient(polynomial, t, k)
    return coefficient

def check_condition_1(x, y):
    r_new = math.floor(compute_r(x, y) +1 )
    n_new = compute_n(x, y)
    k_max = n_new * r_new

    for k in range(r_new, k_max + 1):
        Fx = F(k, r_new, x)
        Fy = F(k, r_new, y)
        if Fx < Fy:
            return "Trumping not possible"
    return "Condition 1 satisfied"

# For checking the second condition
def H1(x):
    result = 0
    for xi in x:
        if xi > 0:  # Avoid log(0) which is undefined
            result += -xi * math.log(xi)
    return result

def check_condition_2(x, y):
    if H1(x) < H1(y):
        return "Trumping not possible"
    return "condition 2 satisfied"

# For checking the third condition

def reciprocal_vector(x, s):
    return [1 / xi**s if xi != 0 else float('inf') for xi in x]

def compute_s(n, x, y):
    log_xmin = math.log(min(x))
    log_ymin = math.log(min(y))
    s = math.log(n) / (log_xmin - log_ymin)
    return s

def check_condition_3(x, y):
    s = compute_s(len(x), x, y)

    for k in range(1, len(x) + 1):
        F_x = F(k, 1, reciprocal_vector(x, s))
        F_y = F(k, 1, reciprocal_vector(y, s))
        if F_x > F_y:
            return "Trumping not possible"

    return "condition 3 satisfied"

# Example usage
x = [0.61, 0.3045, 0.0435, 0.042]
y = [0.7315, 0.1211, 0.1374, 0.01]

#x = [1.22472, 0.611838, 0.0874053, 0.0843912]
#y = [1.32971, 0.220307, 0.24996, 0.0181921]

result = check_condition_1(x, y)
print("Result:", result)

result = check_condition_2(x, y)
print("Result:", result)

result = check_condition_3(x, y)
print("Result:", result)

Result: Condition 1 satisfied
Result: condition 2 satisfied
Result: condition 3 satisfied


# For thermomajorization with thermal states having rational entries.

 Embedding map: The Embedding map, denoted by $\mathcal{E}:P_d \to P_N,\; (N \geq d)$ is defined for an input probability distribution $q:= \{ q_i\}_{i=1}^d$ with support size $d$ and a set of natural numbers $\{\nu_i\}_{i=1}^d$ satisfying $\mathop{\Sigma}\limits_{i=1}^d \nu_i=N$, as:
\begin{equation}
\mathcal{E}(q):=\mathop{\bigoplus}\limits_{i=1}^d q_iu_i
\end{equation}
where $u_i$ is the uniform distribution on the support size $\nu_i$, that is, $u_i=\underbrace{\left\{\frac{1}{\nu_i},\dots,\frac{1}{\nu_i} \right\}}_{\nu_i}$.

For checking the conditions for trumping of a state $\rho$ with the vector of eigenvalues denoted by $q_\rho$ and is block diagonal in the energy eigenbasis with arbitrary accuracy into another block diagonal state $\sigma$ with the vector of eigenvalues denoted by $q_\sigma$ using catalytic thermal operations, given that the vector of eigenvalues of the thermal state $\rho_g$, denoted by $g$, contain all the rational entries, we need to check the same conditions as mentioned above for the state vectors but here $x:=\mathcal{E}(q_\rho), \quad y:=\mathcal{E}(q_\sigma)$, with $\mathcal{E}$ being the embedding map defined above and hence the support size of $x,\;y$ is $N$.

If the thermal state, $g = (\frac{d_1}{D},\frac{d_2}{D},...,\frac{d_n}{D})$ with $D= d_1+d_2+...+d_n$, then the first step is to prepare the list $\mathbf{d} = (d_1,d_2,..., d_n)$. If the initial state vector is $\mathbf q = (q_1,q_2,...,q_n)$, then after the action of the embedding map, the vector will be, $\mathcal{E}(\mathbf q) = (\underbrace{\frac{q_1}{d_1},\dots,\frac{q_1}{d_1}}_{d_1}, \underbrace{\frac{q_2}{d_2},\dots,\frac{q_2}{d_2}}_{d_2}, \dots, \underbrace{\frac{q_n}{d_n},\dots,\frac{q_n}{d_n}}_{d_n},)$. After putting $\mathcal{E}(\mathbf q_{\rho})\equiv \mathbf x$ and $\mathcal{E}(\mathbf q_{\sigma}) \equiv \mathbf y$ in descending order, we need to check the conditions mentioned previously.

In [ ]:
# For thermomajorization with thermal states having rational entries.

from fractions import Fraction
from functools import reduce
from itertools import product
from collections import Counter
from functools import lru_cache
import math

# Given the elements of thermal states as a list of fractions
g = [Fraction(71, 100), Fraction(21, 100), Fraction(6, 100), Fraction(2, 100)]
#g =[Fraction(1, 2), Fraction(1, 4), Fraction(1, 4)]

# Extract denominators from the list of fractions
denominators = [frac.denominator for frac in g]

# Function to calculate the least common multiple (LCM) of two numbers
def lcm(a, b):
    return abs(a * b) // math.gcd(a, b)

# Function to calculate the LCM of a list of numbers
def list_lcm(numbers):
    return reduce(lcm, numbers)

# Calculate the common denominator (LCM of all denominators)
common_denominator = list_lcm(denominators)

# Define updated g

# Convert each elements of thermal state to have the common denominator

# Updated list of g without denominator
def updated_g(g):
    updated_g_numerators = [frac.numerator * (common_denominator // frac.denominator) for frac in g]
    return updated_g_numerators

# For repeating the elements as demanded by Embedding map
def repeat_elements(original_list, repeat_counts):
    return [element for element, count in zip(original_list, repeat_counts) for _ in range(count)]

# The embedding map
def state_after_embedding_map(x,g):
    numer = repeat_elements(x, updated_g(g))
    denom = repeat_elements(updated_g(g), updated_g(g))
    return [m/n for m, n in zip(numer, denom)]


#g = [Fraction(71, 100), Fraction(21, 100), Fraction(6, 100), Fraction(2, 100)]
#x = [0.8634, 0.1301, 0.0059, 0.0006]
#y = [0.9369, 0.0467, 0.0160, 0.0004]
#x = [1.22472, 0.611838, 0.0874053, 0.0843912]
#y = [1.32971, 0.220307, 0.24996, 0.0181921]
y = [0.6098126600657403, 0.3046464157597675, 0.043520852518815774, 0.04202007165567632]
x = [0.7313456157625823, 0.12116969758203461, 0.13747896166533685, 0.010005724990046306]

# x_new and y_new are obtained by applying embedding map to the original states of interests
x_new = state_after_embedding_map(x,g)
y_new = state_after_embedding_map(y,g)

# For rearranging the list in descending order
def descending_order(list_vector):
    list_vector.sort(reverse=True)
    return list_vector

x_new_descending = descending_order(state_after_embedding_map(x,g))
y_new_descending = descending_order(state_after_embedding_map(y,g))

# maximum number of elements of a vector
def compute_n(vector_a, vector_b):
    return max(len(vector_a), len(vector_b))

# For checking the first condition

def compute_r(x, y):
    return math.log(compute_n(x, y)) / (math.log(descending_order(y)[0]) -\
           math.log(descending_order(x)[0]))


# Memorize the factorial function for efficiency
@lru_cache(None)
def factorial(n):
    return math.factorial(n)

# Memorize the product term computation in an efficient way
@lru_cache(None)
def product_term(k_comb, x_group, group_counts):
    result = 1.0
    for i, count in enumerate(group_counts):
        xi = x_group[i]
        ki = k_comb[i]
        result *= (xi**ki) / factorial(ki)
    return result

def F(k, r, x):
    # Group identical elements in x
    x_counter = Counter(x)
    x_group = list(x_counter.keys())
    group_counts = list(x_counter.values())
    n = compute_n(x, y)  # Number of elements in the list

    result = 0.0

    # Generate all possible combinations of (k1, k2, ..., kn) with 1 <= ki <= r
    for k_comb in product(range(1, r + 1), repeat=n):
        if sum(k_comb) == k:
            result += product_term(k_comb, tuple(x_group), tuple(group_counts))

    return result

def check_condition_thermo_1(x, y):
    r_new = math.floor(compute_r(x, y)+1)
    n_new = compute_n(x, y)
    k_max = n_new * r_new

    for k in range(r_new, k_max + 1):
        Fx = F(k, r_new, x)
        Fy = F(k, r_new, y)
        if Fx > Fy:
            return "Trumping not possible"
    return "Condition 1 satisfied"

# For checking the second condition
def H1(x):
    result = 0
    for xi in x:
        if xi > 0:  # Avoid log(0) which is undefined
            result += -xi * math.log(xi)
    return result

def check_condition_thermo_2(x, y):
    if H1(x) > H1(y):
        return "Trumping not possible"
    return "condition 2 satisfied"

# For checking the third condition

def reciprocal_vector(x, s):
    return [1 / xi**s if xi != 0 else float('inf') for xi in x]

def compute_s(n, x, y):
    log_xmin = math.log(min(x))
    log_ymin = math.log(min(y))
    s = max(0,math.floor(math.log(n) / (log_xmin - log_ymin) +1))
    return s

def check_condition_thermo_3(x, y):
    s = compute_s(len(x), x, y)

    for k in range(1, len(x) + 1):
        F_x = F(k, 1, reciprocal_vector(x, s))
        F_y = F(k, 1, reciprocal_vector(y, s))
        if F_x < F_y:
            return "Trumping not possible"

    return "condition 3 satisfied"


thermo_result1 = check_condition_thermo_1(x_new_descending, y_new_descending)
print("Result:", thermo_result1)

thermo_result2 = check_condition_thermo_2(x_new_descending, y_new_descending)
print("Result:", thermo_result2)

thermo_result3 = check_condition_thermo_3(x_new_descending, y_new_descending)
print("Result:", thermo_result3)

Result: Condition 1 satisfied
Result: condition 2 satisfied
Result: condition 3 satisfied


# checking the trumping conditions for thermal states having irrational entries
The following set of finite conditions are sufficient to transform a state $\rho$ with the vector of eigenvalues denoted by $q_\rho$ and is block diagonal in the energy eigenbasis with arbitrary accuracy into another block diagonal state $\sigma$ with the vector of eigenvalues denoted by $q_\sigma$ using catalytic thermal operations, given that the vector of eigenvalues of the thermal state $\rho_g$, denoted by $g$ and containing irrational entries:
\begin{eqnarray*}
F_{k,r}(x) \leq G(k,r)F_{k,r}(y), \qquad r < k \leq Nr  
\end{eqnarray*}
for some non negative real valued function $G:\mathbb{R} \times\mathbb{R}  \to \mathbb{R}_+$, where $x \text{ and }y$ are the probability vectors obtained from $\mathcal{E}(q_\rho) \text{ and }\mathcal{E}(q_\sigma)$ respectively and $\mathcal{E}$ is defined earlier.

Here $G(k,r)$should be such that $\max\limits_k G(k,r) \leq \frac{1}{A(\epsilon)}$ with \
$$\frac{1}{A(\epsilon)}\leq 2^{ -\lVert y \rVert_p^p \left\{ {\left(1+ \frac{\epsilon}{g_{\min}} \right)}^{2p}-1 \right\}}~~~\text{for} p\in (1,r+1)$$
and
$$\frac{1}{A(\epsilon)}\leq 2^{ -\lVert y \rVert_p^p \left\{ 1- {\left(1+ \frac{\epsilon}{g_{\min}} \right)}^{-2p} \right\}}~~~\text{for} p\in (0,1).$$
Above conditions implies,
$$\max\limits_k G(k,r) \leq \min \frac{1}{A(\epsilon)}$$
It is important to note that in the range $p \in (1,r+1)$, the maximum value of $A(\epsilon)$ or in other words the minimum value of $ \frac{1}{A(\epsilon)}$ turns out to be
$2^{ -\lVert y \rVert_1^1 \left\{ {\left(1+ \frac{\epsilon}{g_{\min}} \right)}^{2(r+1)}-1 \right\}}
$ since the value of $\lVert y \rVert_p^p$ is maximized at $p=1$ and $\left\{ {\left(1+ \frac{\epsilon}{g_{\min}} \right)}^{2p}-1 \right\}$ reaches its maximum at $p=r+1$. On the other hand, the minimum value of $\frac{1}{A(\epsilon)}$ can be low as $2^{\left[ - \lVert y \rVert_0^0 \left\{1- \left(1+ \frac{\epsilon}{g_{\min}}\right)^{-2} \right\} \right]}$ since the value of $\lVert y \rVert_p^p$ and $\left\{1- \left(1+ \frac{\epsilon}{g_{\min}}\right)^{-2p} \right\}$ reach their maximum at $p=0$ and $p=1$ respectively in the range $p\in (0,1)$.

In [ ]:
# for checking the trumping conditions for thermal states having irrational entries

from scipy.optimize import minimize_scalar

# Calculate the p-norm of a vector
def p_norm(vector, p):
    norm_sum = sum(abs(x) ** p for x in vector)
    norm = norm_sum ** (1 / p)
    return norm

def reverse_A1(y,eps,g_min,p1,p2):
    return 2**(-(p_norm(y,p1)**p1) * ((1. + eps/g_min)**(2*p2) -1.))

def reverse_A2(y,eps,g_min,p1,p2):
    return 2**(-(p_norm(y,p1)**p1) * (1. - (1. + eps/g_min)**(-2*p2)))

def find_G(x, y, eps, g_min):
    # Minimize reverse_A1 in the range p = 1 to r
    r_new = math.floor(compute_r(x, y))
    min_A1 = reverse_A1(y,eps,g_min,1,r_new)
    # Minimize reverse_A2 in the range p = 0 to 1
    min_A2 = reverse_A2(y,eps,g_min,0,1)

    # Return the overall minimum
    return min(min_A1, min_A2)

def check_condition_thermo_irr_1(x, y, eps, g_min):
    r_new = math.floor(compute_r(x, y) + 1)
    n_new = compute_n(x, y)
    k_max = n_new * r_new

    for k in range(r_new, k_max + 1):
        Fx = F(k, r_new, x)
        Fy = F(k, r_new, y)
        if Fx > find_G(x, y, eps, g_min)* Fy:
            return "Trumping not possible"
    return "Condition 1 satisfied"

y = [0.6098126600657403, 0.3046464157597675, 0.043520852518815774, 0.04202007165567632]
x = [0.7313456157625823, 0.12116969758203461, 0.13747896166533685, 0.010005724990046306]
check_condition_thermo_irr_1(x, y, 0.909209, 0.0192524)


'Condition 1 satisfied'